In [2]:
import os
import math
import numpy as np
import cv2
from PIL import Image ,ImageFilter
import natsort
import xlwt
import numpy as np
import sys
import skimage
import seaborn as sns
import matplotlib.pyplot as plt

# Gray World(while balancing)

In [3]:
def white_balance_loops(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    for x in range(result.shape[0]):
        for y in range(result.shape[1]):
            l, a, b = result[x, y, :]
            l *= 100 / 255.0
            result[x, y, 1] = a - ((avg_a - 128) * (l / 100.0) * 1.1)
            result[x, y, 2] = b - ((avg_b - 128) * (l / 100.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result

# SCB

In [4]:

def apply_mask(matrix, mask, fill_value):
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    return masked.filled()

def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100

    half_percent = percent / 200.0

    channels = cv2.split(img)

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)

        assert len(flat.shape) == 1

        flat = np.sort(flat)

        n_cols = flat.shape[0]

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        thresholded = apply_threshold(channel, low_val, high_val)

        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)



# CLAHE

In [5]:
def Clahe(bgr):
    lab=cv2.cvtColor(bgr,cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1.0,tileGridSize=(40,40))
    l1=clahe.apply(l)
    a1=clahe.apply(a)
    b1=clahe.apply(b)
    bgr=cv2.merge((l1,a,b))
    return cv2.cvtColor(bgr,cv2.COLOR_LAB2BGR)

# Sharpening  

In [8]:
def sharpen(img,kernel=np.array([[0, -1,  0],
                            [-1,  4, -1],                   
                            [0, -1,  0]]),scaled=0.3):
    
    img = img + scaled*cv2.filter2D(src=img, ddepth=-1, kernel=kernel)
    
    return img

In [10]:
#input directory
indirect = "/mnt/1692805e-3445-4084-97bb-231a29d2f2a2/Project Final Year/underwater images/"
#output directory
outdirect = '/mnt/1692805e-3445-4084-97bb-231a29d2f2a2/Project Final Year/test/'
for f in os.listdir(indirect):
    img=cv2.imread(indirect+f)
    img2 = white_balance_loops(img)
    img2 = simplest_cb(img,0.01)
    img2 = Clahe(img2)  
    img2 = sharpen(img=img2)
    cv2.imwrite(outdirect+f,img2)

# THE END